In [1]:
import torch
from torch import nn

# Tokenization

## 공백 기준 토큰화

In [2]:
input_text = "나는 대한민국 서울에 사는 사람이다."
input_text_list = input_text.split()
print(input_text_list)

['나는', '대한민국', '서울에', '사는', '사람이다.']


## 단어집 생성

In [3]:
## Vocabulary 구축.
str2idx = {word:idx for idx, word in enumerate(input_text_list)}
idx2str = {idx:word for idx, word in enumerate(input_text_list)}

print(str2idx)
print(idx2str)

{'나는': 0, '대한민국': 1, '서울에': 2, '사는': 3, '사람이다.': 4}
{0: '나는', 1: '대한민국', 2: '서울에', 3: '사는', 4: '사람이다.'}


## 토큰 리스트 -> id 리스트

In [4]:
input_ids = [str2idx[text] for text in input_text_list]
print(input_ids)

[0, 1, 2, 3, 4]


## 토큰 임베딩

In [5]:
embed_dim = 16
token_embed_layer = nn.Embedding(len(str2idx), embed_dim) ## vocab_size, output_dim

input_tensor = torch.tensor(input_ids)
print(f"Input Tensor : {input_tensor.shape}")

token_embeddings = token_embed_layer(input_tensor)
print(f"Token Embedding : {token_embeddings.shape}")

token_embeddings = token_embeddings.unsqueeze(0)
print(f"Unsqueeze Token Embedding : {token_embeddings.shape}")

Input Tensor : torch.Size([5])
Token Embedding : torch.Size([5, 16])
Unsqueeze Token Embedding : torch.Size([1, 5, 16])


## 위치 임베딩(Absolute Positional Encoding)

In [ ]:
max_position = 12 ## 최대 토큰 수, 최대 문장 길이

ids_len = len(input_ids) ## 시퀀스 데이터가 포함하는 토큰들의 수
print(f"IDs Length : {ids_len}") ## 5

position_ids = torch.arange(ids_len, dtype=torch.long) ## 각 토큰별로 위치값(정수)을 부여.
print(f"Position IDs : {position_ids.shape} {position_ids}") ## (5,) [0, 1, 2, 3, 4]

position_ids = position_ids.unsqueeze(0)
print(f"Unsqueeze Position IDs : {position_ids.shape}") ## (1, 5)

position_embed_layer = nn.Embedding(max_position, embed_dim) ## 각 토큰별 위치(정수형 id)를 embed_dim 차원의 벡터로 임베딩.
position_embeddings = position_embed_layer(position_ids)
print(f"Position Embedding : {position_embeddings.shape}") ## (1, 5, 16)

IDs Length : 5
Position IDs : torch.Size([5]) tensor([0, 1, 2, 3, 4])
Unsqueeze Position IDs : torch.Size([1, 5])
Position Embedding : torch.Size([1, 5, 16])
